<a href="https://colab.research.google.com/github/Joao-Girotto/Desafio-Python-Marvel/blob/main/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalações

In [ ]:
!pip install dotenv

## Importações

In [ ]:
import requests
import hashlib
import os
import time
import pandas as pd
import sqlite3
import json
from google.colab import userdata
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import regex as re
import plotly.express as px

## Pegando as chaves

In [ ]:
PRIVATE_KEY = userdata.get("Private_Key")
PUBLIC_KEY = userdata.get("Public_Key")

## Endpoints e Parâmetros

In [ ]:
ts = str(time.time())
to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()

endCharacters = "https://gateway.marvel.com/v1/public/characters"
endComics = "https://gateway.marvel.com/v1/public/comics"
endSeries = "https://gateway.marvel.com/v1/public/series"
endCreators = "https://gateway.marvel.com/v1/public/creators"
endEvents = "https://gateway.marvel.com/v1/public/events"
endStories = "https://gateway.marvel.com/v1/public/stories"

params = {
    "apikey": PUBLIC_KEY,
    "ts": ts,
    "hash": hash_md5,
    "limit": 10
}

## Pegando o total de requisições

In [ ]:
totalCharacters = requests.get(endCharacters, params=params)
totalCharacters = totalCharacters.json()
total = totalCharacters['data']['total']

totalEvents = requests.get(endEvents, params=params)
totalEvents = totalEvents.json()
totalE = totalEvents['data']['total']

totalCreators = requests.get(endCreators, params=params)
totalCreators = totalCreators.json()
totalC = totalCreators['data']['total']

totalSeries = requests.get(endSeries, params=params)
totalSeries = totalSeries.json()
totalSe = totalSeries['data']['total']

totalComics = requests.get(endComics, params=params)
totalComics = totalComics.json()
totalCo = totalComics['data']['total']

totalStories = requests.get(endStories, params=params)
totalStories = totalStories.json()
totalSt = totalStories['data']['total']

## Função para realizar requisições

In [ ]:
def requisition(endpoint, total, step, limit, params_base=None):
    data_array = []
    offset = 0

    if params_base is None:
        params_base = {}

    while offset <= total:
        params = params_base.copy()
        params['offset'] = offset
        params['limit'] = limit

        response = requests.get(endpoint, params=params)

        if response.status_code != 200:
            print(f"Erro no offset {offset}: status code {response.status_code}")
            offset += step
            continue

        try:
            data = response.json()
        except ValueError:
            print(f"Erro ao converter JSON no offset {offset}")
            offset += step
            continue

        if not data or 'data' not in data or 'results' not in data['data']:
            print(f"Resposta inesperada ou vazia no offset {offset}")
            offset += step
            continue

        data_array.extend(data['data']['results'])
        print(f"Offset: {offset} | Status: {response.status_code}")
        offset += step

    df = pd.DataFrame(data_array)
    return df

## Conversão para CSV

In [ ]:
def create_csv(df, nome_arquivo):  # Já estava praticamente correta

    try:
        df.to_csv(nome_arquivo, index=False, encoding='utf-8')
    except Exception as e:
        print(f'Error when creating the file {nome_arquivo}: {e}')

## Chamando as requisições

In [ ]:
df_characters = requisition(endCharacters, total, step=100, limit=100, params_base=params)
create_csv(df_characters, 'Characters.csv')

# Events
df_events = requisition(endEvents, totalE, step=10, limit=10, params_base=params)
create_csv(df_events, 'Events.csv')

# Creators
df_creators = requisition(endCreators, totalC, step=100, limit=100,  params_base=params)
create_csv(df_creators, 'Creators.csv')

## Realizando Converções

In [ ]:
df_characters = df_characters[['name', 'id', 'description', 'comics', 'thumbnail']]
df_events = df_events[['title', 'id', 'description', 'characters', 'creators', 'comics', 'start', 'end', 'modified']]
df_creators = df_creators[['id', 'firstName', 'middleName', 'lastName', 'fullName', 'suffix', 'thumbnail','comics', 'events', 'stories', 'series']]
df_characters['comics_available'] = df_characters['comics'].apply(lambda x: x.get('available'))
df_characters['comics_returned'] = df_characters['comics'].apply(lambda x: x.get('returned'))
df_characters['thumbnail_path'] = df_characters['thumbnail'].apply(lambda x: x.get('path'))
df_characters['comics'] = df_characters['comics'].apply(lambda x: json.dumps(x))
df_characters['thumbnail'] = df_characters['thumbnail'].apply(lambda x: json.dumps(x))
df_events['comics_available'] = df_events['comics'].apply(lambda x: x.get('available'))
df_events['comics_returned'] = df_events['comics'].apply(lambda x: x.get('returned'))
df_events['comics'] = df_events['comics'].apply(lambda x: json.dumps(x))
df_events['creators_available'] = df_events['creators'].apply(lambda x: x.get('available'))
df_events['creators'] = df_events['creators'].apply(lambda x: json.dumps(x))
df_events['characters_available'] = df_events['characters'].apply(lambda x: x.get('available'))
df_events['characters'] = df_events['characters'].apply(lambda x: json.dumps(x))
df_creators['comics_available'] = df_creators['comics'].apply(lambda x: x.get('available'))
df_creators['events_available'] = df_creators['events'].apply(lambda x: x.get('available'))
df_creators['stories_available'] = df_creators['stories'].apply(lambda x: x.get('available'))
df_creators['series_available'] = df_creators['series'].apply(lambda x: x.get('available'))
df_creators['thumbnail_path'] = df_creators['thumbnail'].apply(lambda x: x.get('path'))
df_creators['comics'] = df_creators['comics'].apply(lambda x: json.dumps(x))
df_creators['events'] = df_creators['events'].apply(lambda x: json.dumps(x))
df_creators['series'] = df_creators['series'].apply(lambda x: json.dumps(x))
df_creators['stories'] = df_creators['stories'].apply(lambda x: json.dumps(x))
df_creators['thumbnail'] = df_creators['thumbnail'].apply(lambda x: json.dumps(x))

## Salvando no Banco de Dados

In [ ]:
con = sqlite3.connect('Marvel.db')
df_characters.to_sql("characters", con, if_exists="replace", index=False)
df_events.to_sql("events", con, if_exists="replace", index=False)
df_creators.to_sql("creators", con, if_exists="replace", index=False)
con.close()

## Salvando Com funções em Banco de Dados

In [ ]:
# Consulta o banco de dados
def query_db(query, params=None, db='Marvel.db'):
  try:
    with sqlite3.connect(db) as con:
      return pd.read_sql_query(query, con, params)

  except Exception as e:
    print(f'Erro com a database: {e}')
    return pd.DataFrame()

# Adiciona os dados à tabela de series
def series_add_db(df):  # Agora recebe um DataFrame

    def calculateDuration(row):
        end = 999 if row['endYear'] == 299 else row['endYear']
        return end - row['startYear']

    def function_dict(x):
        items = x['items'] if isinstance(x, dict) and 'items' in x else []
        creators = [item['name'] for item in items]
        return ', '.join(creators)

    def amount(x):
        return x.get('available', 0)

    with sqlite3.connect('Marvel.db') as con:
        df['duration'] = df.apply(calculateDuration, axis=1)
        df['creator_name'] = df['creators'].apply(function_dict)
        df['character_qty'] = df['characters'].apply(amount)
        df['creator_qty'] = df['creators'].apply(amount)
        df['story_qty'] = df['stories'].apply(amount)
        df['event_qty'] = df['events'].apply(amount)
        df['comic_qty'] = df['comics'].apply(amount)

        df = df[['id', 'title', 'description', 'startYear', 'endYear',
                 'rating', 'type', 'duration', 'creator_name', 'creator_qty',
                 'character_qty', 'story_qty', 'event_qty', 'comic_qty']]

        df.to_sql("series", con, if_exists="append", index=False)

# Adiciona os dados à tabela de comics
def comics_add_db(df):  # Agora recebe um DataFrame

    def funcao_series(x):
        return x.get('name', '') if isinstance(x, dict) else ''

    def funcao_price(x):
        if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict):
            return x[0].get('price', '')
        return ''

    with sqlite3.connect('Marvel.db') as con:
        df['price'] = df['prices'].apply(funcao_price)
        df['series_name'] = df['series'].apply(funcao_series)

        df = df[['id', 'digitalId', 'title', 'issueNumber', 'description',
                 'format', 'pageCount', 'series_name', 'price']]

        df.to_sql("comics", con, if_exists="append", index=False)


# Adiciona os dados à tabela de stories
def stories_add_db(df):  # Agora recebe um DataFrame
    with sqlite3.connect('Marvel.db') as con:
        df = df[['id', 'title', 'description', 'type']]
        df.to_sql("stories", con, if_exists="append", index=False)

## Adicionando mais Endpoints ao Banco

In [ ]:
df_series = requisition(endSeries, totalSe, step=100, limit=100, params_base=params)
create_csv(df_series, nome_arquivo='Series.csv')
series_add_db(df_series)

df_comics = requisition(endComics, total, step=100, limit=100, params_base=params)
create_csv(df_comics, nome_arquivo='Comics.csv')
series_add_db(df_comics)

df_stories = requisition(endStories, total, step=100, limit=100, params_base=params)
create_csv(df_stories, nome_arquivo='Stories.csv')
series_add_db(df_stories)

## Consultando o Banco de Dados

In [ ]:
con2 = sqlite3.connect('Marvel.db')

df2 = pd.read_sql_query("SELECT id, name, description, comics, comics_available, comics_returned FROM characters", con2)

query1 = """
SELECT SUBSTR(name, 1, 1) AS letra_inicial, COUNT(*) AS total_por_letra
FROM characters
GROUP BY letra_inicial
ORDER BY letra_inicial;
"""
query2 = """
SELECT id, name, comics_available
FROM characters
ORDER BY comics_available DESC
LIMIT 10;
"""

query3 = """
SELECT id, name, description, comics_available
FROM characters
WHERE description IS NOT NULL AND description != '';
"""

query3_1 = """
SELECT
  COUNT (*) FILTER (WHERE description IS NOT NULL AND description != '') AS com_descricao,
  COUNT (*) FILTER (WHERE description IS NULL OR description = '') AS sem_descricao
FROM characters;
"""

query4 = """
SELECT title, comics_available
FROM events
ORDER BY comics_available DESC
LIMIT 10;
"""

query5 = """
SELECT title,
       strftime('%d/%m/%Y', start) AS start_formatted,
       strftime('%d/%m/%Y', end) AS end_formatted,
       CAST((julianday(end) - julianday(start)) / 365.25 AS INTEGER) AS duration_in_years,
       comics_available
FROM events
WHERE julianday(start) <= julianday('2025-05-16') AND julianday(end) <= julianday('2025-05-16')
ORDER BY julianday(end) - julianday(start) DESC;
"""

query6 = """
SELECT title,
       strftime('%d/%m/%Y', SUBSTR(modified, 1, 10)) AS modified,
       strftime('%d/%m/%Y', start) AS start,
       strftime('%d/%m/%Y', end) AS end
FROM events
ORDER BY SUBSTR(modified, 1, 4) DESC;
"""

query7 = """
SELECT title, characters_available, comics_available
FROM events
ORDER BY characters_available DESC
LIMIT 10;
"""

query8 = """
SELECT title, characters_available, creators_available, comics_available
FROM events
WHERE characters_available > 0 AND creators_available > 0 AND comics_available > 0;
"""


query9 = """
SELECT  firstName, middleName, lastName, fullName, comics_available, events_available
FROM creators
ORDER BY comics_available DESC
LIMIT 15;
"""

query10 = """
SELECT
  COUNT(*) FILTER (WHERE thumbnail_path NOT LIKE '%image_not_available') AS Criadores_Com_Imagem_Associada,
  COUNT(*) FILTER (WHERE thumbnail_path LIKE '%image_not_available') AS Criadores_Sem_Imagem_Associada
FROM creators;
"""

query11 = """
SELECT
  COUNT(*) FILTER (WHERE thumbnail_path NOT LIKE '%image_not_available') AS Personagenss_Com_Imagem,
  COUNT(*) FILTER (WHERE thumbnail_path LIKE '%image_not_available') AS Personagens_Sem_Imagem
FROM characters;
"""

count_letters = pd.read_sql_query(query1, con2)
most_comics_charatcers = pd.read_sql_query(query2, con2)
description_characters = pd.read_sql_query(query3, con2)
most_events_comics = pd.read_sql_query(query4, con2)
most_duration_events = pd.read_sql_query(query5, con2)
events_modified = pd.read_sql_query(query6, con2)
events_most_characters = pd.read_sql_query(query7, con2)
count_description = pd.read_sql_query(query3_1, con2)
comics_per_events = pd.read_sql_query(query9, con2)
creators_images = pd.read_sql_query(query10, con2)
characters_images = pd.read_sql_query(query11, con2)


con2.close()

# Funções Auxiliares

In [ ]:
def plot_donut_charts(df, colors=None, figsize=(10, 6), center_text_prefix='Total'):
    # Transpor o DataFrame para que cada coluna vire um gráfico de donut
    df_transposed = df.transpose()

    # Criar gráficos de "donut"
    axes = df_transposed.plot.pie(
        subplots=True,
        autopct='%1.1f%%',
        figsize=figsize,
        legend=False,
        startangle=90,
        colors=colors
    )

    # Adicionar círculo no centro e texto com o total de cada gráfico
    for ax, col in zip(axes, df_transposed.columns):
        total = df_transposed[col].sum()
        centre_circle = plt.Circle((0, 0), 0.70, fc='white')
        ax.add_artist(centre_circle)

        # Texto central com total
        ax.text(0, 0, f'{center_text_prefix}:\n{total}', fontsize=10,
                fontweight='bold', ha='center', va='center')

        ax.set_ylabel('')  # Remove rótulo do eixo y
        ax.set_title(col, fontsize=12, fontweight='bold')

    plt.tight_layout()
    plt.show()

def verify_first_letters(nome_completo):
  partes_nome = nome_completo.split()
  if len(partes_nome) >= 2:
    nome = partes_nome[0]
    sobrenome = partes_nome[-1] # Pega a última parte como sobrenome
    if nome and sobrenome: # Verifica se nome e sobrenome não são vazios
      return nome[0].lower() == sobrenome[0].lower()
  return False # Retorna False se não for possível verificar (nome curto ou vazio)

def plot_side_by_side_donuts(series1, series2,
                            titles=None,
                            colors=None,
                            figsize=(12, 6),
                            donut_width=0.3,
                            center_text_prefix='Total'):
    fig, axes = plt.subplots(1, 2, figsize=figsize)

    # Donut 1
    series1.plot.pie(
        ax=axes[0],
        autopct='%1.1f%%',
        startangle=90,
        colors=colors,
        legend=False,
        wedgeprops={'width': donut_width, 'edgecolor': 'white'}
    )
    # Adiciona círculo no centro do primeiro donut
    centre_circle1 = plt.Circle((0, 0), 1 - donut_width, fc='white')
    axes[0].add_artist(centre_circle1)
    total1 = series1.sum()
    axes[0].text(0, 0, f'{center_text_prefix}:\n{total1}', fontsize=10,
                 fontweight='bold', ha='center', va='center')

    # Donut 2
    series2.plot.pie(
        ax=axes[1],
        autopct='%1.1f%%',
        startangle=90,
        colors=colors,
        legend=False,
        wedgeprops={'width': donut_width, 'edgecolor': 'white'}
    )
    # Adiciona círculo no centro do segundo donut
    centre_circle2 = plt.Circle((0, 0), 1 - donut_width, fc='white')
    axes[1].add_artist(centre_circle2)
    total2 = series2.sum()
    axes[1].text(0, 0, f'{center_text_prefix}:\n{total2}', fontsize=10,
                 fontweight='bold', ha='center', va='center')

    # Títulos
    if titles:
        axes[0].set_title(titles[0], fontsize=12, fontweight='bold')
        axes[1].set_title(titles[1], fontsize=12, fontweight='bold')

    # Remover rótulos dos eixos y
    axes[0].set_ylabel('')
    axes[1].set_ylabel('')

    plt.tight_layout()
    plt.show()

# Plota um gráfico, wrapper do plot do pandas, objetivo é a padronização dos gráficos
def plot(dataframe,
         title,
         x_label,
         y_label,
         grid='y',
         kind='bar',
         color='purple',
         rotation=90,
         show=True,
         **kwargs):
  ax = dataframe.plot(kind=kind, color=color, title=title, **kwargs)
  ax.set_xlabel(x_label)
  ax.set_ylabel(y_label)
  plt.xticks(rotation=rotation)
  plt.tight_layout()
  plt.grid(axis=grid)

  if show:
    plt.show()

  return ax

# Funções Series

## Função para analisar a quatidade de séries por ano

In [ ]:
# Analisa a quantidade de series por ano e retorna um dataframe
def series_by_year(dataframe, show_plot=False):
  series_year = (
      dataframe[(dataframe['startYear'] > 0) &
                (dataframe['startYear'] < 2029)]
                ['startYear']
                .value_counts()
                .sort_index()
      )

  if show_plot:
    plot(
      dataframe=series_year,
      title='Séries por Década',
      x_label='Década',
      y_label='Quantidade de Séries',
      figsize=(20, 6)
    )

  return series_year

## Função para analisar a média de duração das séries

In [ ]:
# Analisa a média de duração das series e retorna um dataframe
def series_average_duration(dataframe, show_plot=False):
  anos_series = (
      dataframe[(dataframe['duration'] >= 0) &
                (dataframe['duration'] < 999)]
                ['duration']
                .value_counts()
                .sort_index()
                .head(10)
      )

  if show_plot:
    plot(
      dataframe=anos_series,
      title='Series: Média da duração das series',
      x_label='Duração (anos)',
      y_label='Quantidade de séries'
   )

  media = (
      dataframe[(dataframe['duration'] >= 0) &
                (dataframe['duration'] < 999)]
                ['duration']
                .mean()
      ) ## 999 definido para séries em produção
  return anos_series, media

## Função para analisar a distribuição das classificações

In [ ]:
# Analisa a distribuição das classificações etárias de series
# e retorna um dataframe
def series_distribution_rating(dataframe, show_plot=False):
  media = dataframe[dataframe['rating'] != '']['rating'].value_counts()

  if show_plot:
    plot(
      dataframe=media,
      x_label='Classificação etária',
      y_label='Quantidade de séries',
      title='Distribuição classificatória de faixa etária'
    )

  return media

## Função para analisar a distribuição dos tipos

In [ ]:
# Analisa a distribuição dos tipos de series e retorna um dataframe
def series_distribuition_type(dataframe, show_plot=False):
  series_type = dataframe[dataframe['type'] != '']['type'].value_counts()

  if show_plot:
    plot(
      dataframe=series_type,
      x_label='Tipo',
      title='Distribuição dos tipos de séries',
      y_label='Quantidade de séries'
    )

  return series_type

## Função para analisar a quantidade de personagens por ano

In [ ]:
# Analisa a quantidade de personagens por ano de series e retorna um dataframe
def series_quantity_characters_year(dataframe, show_plot=False):
  tempo = (
      dataframe[
          (dataframe['character_qty'] > 0) &
          (dataframe['startYear'] > 1900) &
          (dataframe['startYear'] < 2029)]
          .groupby('startYear')
          ['character_qty']
          .sum()
        )

  falta_anos = pd.Series(0, index=range(1939, 2025))
  arrumado =  falta_anos.add(tempo, fill_value=0)

  if show_plot:
    plot(
        dataframe=arrumado,
        title='Quantidade de personagens por ano',
        x_label='Ano',
        y_label='Quantidade de personagens',
        figsize=(20, 6)
    )

  return arrumado

# Funções Comics

## Função para calcular a quatidade de cada formato

In [ ]:
# Calcula a quantidade de cada formato de comics e retorna um dataframe
def comics_count_format(dataFrame, show_plot=False):
  format = dataFrame[dataFrame['format'] != '']['format'].value_counts()

  if show_plot:

    fig = px.bar(
        format,
        x=format.index,
        y=format.values,
        labels={'x': 'Formato', 'y': 'Quantidade de comics'},
        title='Quantidade de cada tipo',
        color_discrete_sequence=['purple']
    )

    fig.update_layout(
        xaxis_title='Formato',
        yaxis_title='Quantidade de comics',
        xaxis_tickangle=-45
    )

    fig.show()

  return format

## Função para calcular a média de preço das comics sobre o tempo

In [ ]:
# Calcula a média de preço das comics sobre o tempo e retorna um dataframe
def comics_average_price_over_time(dataframe, show_plot=False):
  def extract_number(text):
    match = re.findall(r'\d+', text)
    return int(match[0]) if match else None

  dataframe['year'] = dataframe['title'].apply(extract_number)
  price_over_time = dataframe[['year', 'price']]

  price_over_time = price_over_time[(price_over_time['year'] > 1900) &
                                    (price_over_time['year'] < 2050)]

  price_filter = price_over_time.query('price > 0 and price < 20').copy()
  price_filter = price_filter.dropna()
  price_filter_average = price_filter.groupby('year').mean()

  if show_plot:
    plt.figure(figsize=(20, 6))

    plt.plot(
        price_filter_average.index,
        price_filter_average['price'],
        color='purple',
        label='Preço Médio'
    )

    plt.scatter(
        price_filter['year'],
        price_filter['price'],
        color='grey',
        label='Preços individuais',
        alpha=0.4,
        s=10
    )

  return price_filter_average

## Função para calcular a média de páginas das comics sobre o tempo

In [ ]:
# Calcula a média de páginas das comics sobre o tempo e retorna um dataframe
def comics_average_pages_over_time(dataframe, show_plot=False):
  def extract_number(text):
    match = re.findall(r'\d+', text)
    return int(match[0]) if match else None

  dataframe['year'] = dataframe['title'].apply(extract_number)
  pages_over_time = dataframe[['year', 'pageCount']]

  pages_over_time = pages_over_time[(pages_over_time['year'] > 1900) &
                                    (pages_over_time['year'] < 2050)]

  pages_filter = pages_over_time.query('pageCount > 0 and pageCount < 200')
  pages_filter = pages_filter.dropna()
  pags_filter_average = pages_filter.groupby('year').mean()

  if show_plot:

    plot(
      dataframe=pags_filter_average,
      title='Média de páginas',
      x_label='Ano',
      y_label='Média de páginas',
      kind='line'
    )

  return pags_filter_average

# Funções Stories

## Função para calcular a quantidade de cada tipo de stories

In [ ]:
# Calcula a quantidade de cada tipo de stories e retorna um dataframe
def stories_quantity_type(dataframe, show_plot=False):
  types = dataframe[dataframe['type'] != '']
  clean_types = types['type'].value_counts()
  if show_plot:

    fig = px.bar(
        clean_types,
        x=clean_types.index,
        y=clean_types.values,
        labels={'x': 'Tipo', 'y': 'Quantidade'},
        title='Quantidade de cada tipo',
        color_discrete_sequence=['purple']
    )

    fig.update_layout(
        xaxis_title='Tipos',
        yaxis_title='Quantidade',
        xaxis_tickangle=-45
    )

    fig.show()

  return types

## Selecionando do Banco de Dados

In [ ]:
dataSeries = query_db('SELECT * FROM series')
dataComics = query_db('SELECT * FROM comics')
dataStories = query_db('SELECT * FROM stories')

# Characters

## INSIGHT 1 Characters - Criando função para selecionar entidades que tenham nome aliterativo

In [ ]:
# Aplica a função à coluna 'name' e cria uma nova coluna booleana
dataframe = df2.copy()
dataframe['nome_sobrenome'] = df2['name'].apply(verify_first_letters)

# Para ver os personagens que atendem à condição
df_same_letter = dataframe[dataframe['nome_sobrenome']]
df_same_letter[['name', 'nome_sobrenome']]

## INSIGHT 2 Characters - Número de entidades por letra

In [ ]:
count_letters

## INSIGHT 2 Characters - Gráfico

In [ ]:
count_letters.plot.bar(x='letra_inicial', y='total_por_letra', legend=False)

## INSIGHT 3 Characters - Demonstração dos 10 personagens que tem mais quadrinhos

In [ ]:
most_comics_charatcers.plot.bar(x='name', y='comics_available')

## INSIGHT 4 Characters - Personagens que tem descrição

In [ ]:
description_characters

## INSIGHT 4 Characters - Gráfico de comparação da quantidade de personagens com descrição

In [ ]:
plot_donut_charts(count_description)

## INSIGHT 5 Characters - Gráfico de personagens que tem imagens

In [ ]:
plot_donut_charts(characters_images)

In [ ]:
plot_side_by_side_donuts(
    count_description.iloc[0],
    characters_images.iloc[0],
    titles=["Distribuição de Descrição", "Distribuição de Imagem"],
    colors=['#66c2a5', '#fc8d62'],
    donut_width=0.2
)

# Events

## INSIGHT 1 Events - Os 10 eventos com mais histórias em quadrinhos

In [ ]:
most_events_comics.plot.bar(x='title', y='comics_available')
# plot(most_events_comics, "Teste", x_label=most_events_comics['title'], y_label='comics_available', )

## INSIGHT 2 Events - Os eventos com maior duração

In [ ]:
most_duration_events

## INSIGHT 2 Events - Gráfico de demonstração de duração dos eventos

In [ ]:
#df9
df_timeline = events_modified.copy()

# Converter datas de string para datetime
df_timeline['start'] = pd.to_datetime(df_timeline['start'], format='%d/%m/%Y', errors='coerce')
df_timeline['end'] = pd.to_datetime(df_timeline['end'], format='%d/%m/%Y', errors='coerce')

# Remover eventos com datas inválidas
df_timeline = df_timeline.dropna(subset=['start', 'end'])

# Ordenar por data de início
df_timeline = df_timeline.sort_values(by='start')
plt.figure(figsize=(12, len(df_timeline) * 0.14))  # ajusta altura dinamicamente
plt.margins(y=0)

# Índice para os nomes dos eventos
y_positions = range(len(df_timeline))

# Plotar as barras horizontais
plt.barh(
    y=y_positions,
    width=(df_timeline['end'] - df_timeline['start']).dt.days,
    left=df_timeline['start'],
    color='skyblue'
)

# Nome dos eventos no eixo Y
plt.yticks(ticks=y_positions, labels=df_timeline['title'])

# Formatando o eixo X para mostrar ano
plt.gca().xaxis_date()
plt.xlabel('Ano')
plt.title('Linha do Tempo dos Eventos Marvel')
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

## INSIGHT 3 Events - Eventos modificados mais recentemente

In [ ]:
events_modified

## INSIGHT 4 Events - Distribuição de Personagens por Duração do Evento dos 10 Eventos mais longos

In [ ]:
plt.figure(figsize=(10, 6))  # ajusta altura dinamicamente
scatter = plt.scatter(
    x=events_most_characters['characters_available'],
    y=events_most_characters['title'],
    s=events_most_characters['comics_available'] * 2,  # tamanho da bolha (ajuste o fator)
    c=events_most_characters['comics_available'],  # cor baseada em quadrinhos
    cmap='viridis',
    alpha=0.7,
    edgecolors='black'
)

plt.xlabel('Personagens Disponíveis')
plt.ylabel('Titulo do Evento')
plt.title('Escala dos Eventos Marvel: Relação entre Personagens e Quadrinhos')
plt.colorbar(scatter, label='Quadrinhos Disponíveis')
plt.grid(True)
plt.tight_layout()
plt.show()

# Creators

## INSIGHT 1 Creators - Distribuição entre quantos comics o autor escreveu e em quantos eventos estava

In [ ]:
plt.figure(figsize=(10, 8))

scatter = plt.scatter(
    x=comics_per_events['events_available'],
    y=comics_per_events['fullName'],
    s=comics_per_events['comics_available'],  # tamanho da bolha
    c=comics_per_events['comics_available'],      # cor baseada nos quadrinhos
    cmap='plasma',
    alpha=0.7,
    edgecolors='black'
)

plt.xlabel('Eventos Disponíveis')
plt.ylabel('Nome do Criador')
plt.title('Criadores: Participação em Eventos vs Quadrinhos (Tamanho/Cor)')
plt.colorbar(scatter, label='Quadrinhos Disponíveis')
plt.grid(True)
plt.tight_layout()
plt.show()

## INSIGHT 2 Creators - Comparação de Creators com imagens associadas e sem imagens associadas

In [ ]:
plot_donut_charts(creators_images)

# Comics

## INSIGHT 1 Comics: Média de preço sobre o tempo

In [ ]:
comics_average_price_over_time(dataComics, show_plot=True)

## INSIGHT 2 Comics: Média de páginas sobre o tempo

In [ ]:
comics_average_pages_over_time(dataComics, show_plot=True)

## INSIGHT 3 Comics: Os formatos mais comuns

In [ ]:
comics_count_format(dataComics, show_plot=True)

# Series

## INSIGHT 1 Series: Quantidade de issues lançadas por ano

In [ ]:
series_by_year(dataSeries, show_plot=True)

## INSIGHT 2 Series: Média da duração das series

In [ ]:
series_average_duration(dataSeries, show_plot=True)

## INSIGHT 3 Series: Distribuição classificatória de faixa etária

In [ ]:
series_distribution_rating(dataSeries, show_plot=True)

## INSIGHT 4 Series: Distribuição dos tipos de séries

In [ ]:
series_distribuition_type(dataSeries, show_plot=True)

## INSIGHT 5 Series: Quantidade de personagens por ano

In [ ]:
series_quantity_characters_year(dataSeries, show_plot=True)

## Dados errados da API em relação a duração

In [ ]:
duration = dataSeries['duration'].value_counts().sort_index()
dataSeries[(dataSeries['duration'] < 0) | (dataSeries['duration'] > 999)]

# Stories

## INSIGHT 1 Stories: Quantidade de cada tipo

In [ ]:
types = stories_quantity_type(dataStories, show_plot=True)